# Dijkstras 

It runs on directed and weighted graph. It is similar to prims and BFS. It is similar to BFS because it uses min priority queue. Similar to Prims because it uses the growing tree concept.



In [1]:
import math
import sys

class Vertex:
    def __init__(self, node, weight=0):
        self.id = node
        self.visited = False
        self.adjList = {}
        self.weight = weight
        self.distance = sys.maxsize
        self.color = 'WHITE'
        self.parent = None
        self.start_time = -1
        self.finish_time = -1

    def set_start_time(self, start_time):
        self.start_time = start_time

    def get_start_time(self):
        return self.start_time

    def get_finish_time(self):
        return self.finish_time

    def set_finish_time(self, finish_time):
        self.finish_time = finish_time

    def add_neighbour(self, node, weight=0):
        self.adjList[node] = weight

    def set_distance(self, distance):
        self.distance = distance

    def get_distance(self):
        return self.distance

    def set_visited(self):
        self.visited = True

    def get_visited(self):
        return self.visited

    def set_id(self, node):
        self.id = node

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjList[neighbor]

    def get_neighbours(self):
        return self.adjList

    def set_parent(self, parent):
        self.parent = parent

    def get_parent(self):
        return self.parent

    def set_color(self, color):
        self.color = color

    def get_color(self):
        return self.color


class GraphAL():
    def __init__(self, isDag=False):
        self.numVertices = 0
        self.isDag = isDag
        self.vertices = {}

    def add_vertex(self, ID):
        self.numVertices += 1
        new_vertex = Vertex(ID)
        self.vertices[ID] = new_vertex

    def add_edge(self, u, v, weight):
        if u not in self.vertices:
            self.add_vertex(u)

        if v not in self.vertices:
            self.add_vertex(v)

        self.vertices[u].add_neighbour(self.vertices[v], weight)

        if not self.isDag:
            self.vertices[v].add_neighbour(self.vertices[u], weight)

    def get_vertex(self, node):
        if node in self.vertices:
            return self.vertices[node]

        return -1

    def get_vertices(self):
        return self.vertices

    def get_edges(self):
        edges = []
        for u in self.vertices.values():
            for v in u.get_neighbours():
                edges.append((u.get_id(), v.get_id(), u.get_weight(v)))

        return (edges)


G = GraphAL(True)
G.add_vertex('a')
G.add_vertex('b')
G.add_vertex('c')
G.add_vertex('d')
G.add_vertex('e')

G.add_edge('a', 'e', 10)
G.add_edge('a', 'c', 20)
G.add_edge('c', 'b', 30)
G.add_edge('b', 'e', 40)
G.add_edge('e', 'd', 50)
G.add_edge('f', 'e', 60)

print(G.get_vertices())
G.get_edges()

{'a': <__main__.Vertex object at 0x109150b38>, 'b': <__main__.Vertex object at 0x109150b70>, 'c': <__main__.Vertex object at 0x1091506a0>, 'd': <__main__.Vertex object at 0x109150320>, 'e': <__main__.Vertex object at 0x1091502b0>, 'f': <__main__.Vertex object at 0x109150588>}


[('a', 'e', 10),
 ('a', 'c', 20),
 ('b', 'e', 40),
 ('c', 'b', 30),
 ('e', 'd', 50),
 ('f', 'e', 60)]

In [2]:
class MinHeap():
    def __init__(self):
        self.minHeap = []
        self.hashmap = {}
        self.size = 0

    def get_size(self):
        return self.size

    def get_min(self):
        if self.minHeap:
            return self.minHeap[0]

    def percolate_up(self, index):
        # If the parent is larger than the element at index then swap and update the hashmap

        while (index - 1) // 2 >= 0:
            if (self.minHeap[index])[1] < (self.minHeap[(index - 1) // 2])[1]:
                # update the indexes in the map
                self.hashmap[(self.minHeap[index])[0]] = (index - 1) // 2
                self.hashmap[(self.minHeap[(index - 1) // 2])[0]] = index

                # swap the elements
                self.minHeap[index], self.minHeap[
                    (index - 1) // 2] = self.minHeap[(index - 1) //
                                                     2], self.minHeap[index]

            index = (index - 1) // 2

    def get_min_child(self, i):
        if 2 * i + 2 > self.size - 1:
            return 2 * i + 1

        if (self.minHeap[2 * i + 1])[1] < (self.minHeap[2 * i + 2])[1]:
            return 2 * i + 1
        else:
            return 2 * i + 2

    def percolate_down(self, i):
        while 2 * i + 1 < self.size:
            index = self.get_min_child(i)
            # update indexes in hashmap
            self.hashmap[(self.minHeap[i])[0]] = index
            self.hashmap[(self.minHeap[index][0])] = i

            # swap the minheap array indexes
            self.minHeap[i], self.minHeap[index] = self.minHeap[
                index], self.minHeap[i]

            i = index

    def insert(self, data):
        # add the element to the end of the array and percolate up
        self.size += 1
        self.hashmap[data[0]] = self.size - 1
        self.minHeap.append(data)

        self.percolate_up(self.size - 1)

    def contains(self, key):
        if key in self.hashmap:
            return True

    def decrease_key(self, key, value):
        if self.contains(key):
            index = self.hashmap.get(key)
            (self.minHeap[index]) = (key, value)
            self.percolate_up(index)

    def delete_min(self):
        self.print_heap()
        if self.contains((self.minHeap[0])[0]):
            key = self.minHeap[self.size - 1]
            print(key)
            self.hashmap[key[0]] = 0
            del self.hashmap[(self.minHeap[0])[0]]

        temp = (self.minHeap[0])
        # copy the last element in the min heap to the front
        self.minHeap[0] = self.minHeap[self.size - 1]
        del self.minHeap[self.size - 1]
        self.size -= 1
        if self.size > 1:
            self.percolate_down(0)
        return temp

    def heapify(self, A):
        i = (len(A) // 2) - 1
        self.size = len(A)
        self.minHeap = A
        for i in range(len(self.minHeap)):
            self.hashmap[(self.minHeap[i])[0]] = i
        while i >= 0:
            self.percolate_down(i)

            i -= 1
            
        return self.minHeap

    def print_heap(self):
        print(self.minHeap)
        print(self.hashmap)



In [3]:
def relax(u, v):
    
    if v.get_distance()> u.get_distance()+ u.get_weight(v):
        
        v.set_distance(u.get_distance()+ u.get_weight(v))
        v.set_parent(u)



def dijkstras(G, s):
    arr = [(s, 0)]
    for u in G.get_vertices().values():
        if u.get_id() not in arr[0]:
            arr.append((u.get_id(), sys.maxsize))
    print(arr)
    m = MinHeap()
    a = m.heapify(arr)
    print(arr)
    print(a)

    while m.get_size()!=0:
        u = m.delete_min(a)
        print(u)
        for v in G.get_vertex(u[0]).get_neighbours():
            relax(G.get_vertex(u[0]), v)


dijkstras(G, 'a')

[('a', 0), ('b', 9223372036854775807), ('c', 9223372036854775807), ('d', 9223372036854775807), ('e', 9223372036854775807), ('f', 9223372036854775807)]
[('f', 9223372036854775807), ('e', 9223372036854775807), ('c', 9223372036854775807), ('d', 9223372036854775807), ('b', 9223372036854775807), ('a', 0)]


NameError: name 'heapq' is not defined

In [7]:
q = MinHeap()
print(q.heapify([('a', 9223372036854775807), ('b', 9223372036854775807), ('c', 0)]))

[('c', 0), ('b', 9223372036854775807), ('a', 9223372036854775807)]
